In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install mtcnn

In [ ]:
# SE model has dense layers 512-64-2
from SE_Xception_GAP  import *
# from ELBP_Functions     import *
from ELBP_Custom_Layers import *

from Parameter_Calc import *


In [ ]:
from __future__ import print_function
from __future__ import absolute_import

from keras.preprocessing import image
import matplotlib.pyplot as plt
import time
from tqdm.notebook import tqdm as log_progress
import pandas as pd
import numpy as np
import time
import random
import os
from matplotlib import pyplot as plt
import cv2
import tensorflow as tf

import warnings
import numpy as np

from keras.preprocessing import image
from keras.utils import data_utils
from keras.models import Model
from keras import layers
from keras.models import Sequential
from keras.layers import *
import os
import pandas  as pd
from keras.utils.vis_utils import plot_model

#from keras.engine.topology import get_source_inputs
from keras.utils.data_utils import get_file
from keras import backend as K
from keras.applications.imagenet_utils import decode_predictions
from tensorflow.python.keras.layers import Layer, InputSpec
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.utils.vis_utils import plot_model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades +    'haarcascade_frontalface_default.xml')
from mtcnn.mtcnn import MTCNN
from keras.layers.merge import concatenate
from keras.layers import BatchNormalization
from sklearn.decomposition import PCA

In [ ]:

# Helper Funciton to get Model Photo
def model_photu(model,path='model_plot.png',n=True,expand=False,direct='TB',shapes=True):
   return plot_model(model, to_file=os.getcwd()+path, show_shapes=shapes, show_layer_names=n,expand_nested=expand,rankdir=direct)

In [ ]:
# Dataset imports and Extraction

!7z x '/content/drive/MyDrive/ML_Data/Datasets_Preprocessed/replay_data_every_10th.zip' -o'/content/'
!7z x '/content/drive/MyDrive/ML_Data/Datasets_Preprocessed/CASIA_FASD_FRAMES.zip' -o'/content/'
!7z x '/content/drive/MyDrive/ML_Data/Datasets_Preprocessed/MSU-MFSD_FRAMES.zip' -o'/content/MSU-MFSD_FRAMES/'


In [ ]:

#Folders
Replay_train_folder = '/content/drive/MyDrive/ML_Data/Datasets_Preprocessed/replay_data_every_10th/train'
Replay_val_folder   = '/content/drive/MyDrive/ML_Data/Datasets_Preprocessed/replay_data_every_10th/dev'
Casia_train_folder  = '/content/drive/MyDrive/ML_Data/Datasets_Preprocessed/CASIA_FASD_FRAMES/train_release'
Casia_val_folder    = Casia_train_folder
Oulu_train_folder   = '/content/drive/MyDrive/ML_Data/Datasets_Preprocessed/OULU_frames/Train_Files'
Oulu_val_folder     = '/content/drive/MyDrive/ML_Data/Datasets_Preprocessed/OULU_frames/Dev_Files_frames'


replay_test_folder  =  '/content/drive/MyDrive/ML_Data/Datasets_Preprocessed/replay_data_every_10th/test'
casia_test_folder   =  '/content/drive/MyDrive/ML_Data/Datasets_Preprocessed/CASIA_FASD_FRAMES/test_release'

#Labels
Replay_train_full   = pd.read_csv('/content/drive/MyDrive/ML_Data/Labels/Replay_train_rgb.csv')
Replay_dev_full     = pd.read_csv('/content/drive/MyDrive/ML_Data/Labels/Replay_dev_rgb.csv')
Casia_train_full    = pd.read_csv('/content/drive/MyDrive/ML_Data/Labels/CASIA_FASD_TRAIN.csv')
Casia_dev_full      =Casia_train_full.iloc[2::3]
Casia_train_full    = Casia_train_full[Casia_train_full.index % 3 != 0]

#generators
train_gen = ImageDataGenerator(rescale=1./255,horizontal_flip=True)
val_gen   = ImageDataGenerator(rescale=1./255,horizontal_flip=True)
test_gen  = ImageDataGenerator(rescale=1./255)

# BASE MODEL - SQUEEZE-XCEPTION BOTH INTRA AND CROSS WEIGHTS
# Fine Tuned Modifieid Exception models 

Replay_SE_Xception_GAP_intra          ='/content/drive/MyDrive/ML_Data/SavedModels/Xcept_GAP_Replay_9.hdf5'
Casia_SE_Xception_GAP_intra           ='/content/drive/MyDrive/ML_Data/SavedModels/Xception_Casia_10_BN.hdf5'
MSU_SE_Xception_GAP_intra             ='/content/drive/MyDrive/ML_Data/SavedModels/Xcept_GAP_MSU_8.hdf5'




In [ ]:
#check every =2 will check half data set, 3 will chcek one third, 4 will check 25%

def TestModel(l_model,testOn,filename='result.csv',check_every=1):
     if(testOn=='Replay'):
      print('Loading Replay Samples')
      test_full = pd.read_csv('/content/drive/MyDrive/ML_Data/Labels/Replay_Test_rgb.csv')
      test_folder=replay_test_folder
      if os.path.exists('/content/replay_data_every_10th'):
        print('Found Local Files')
        test_folder='/content/replay_data_every_10th/test'
        test_full['filename'] = test_full['filename'].str.replace('drive/MyDrive/ML_Data/Datasets_Preprocessed/','')
      if check_every >1 :
        test_full=test_full.iloc[::check_every]
      test_full=test_full.dropna()

      testData = test_gen.flow_from_dataframe(dataframe =test_full,directory = test_folder,  x_col = "filename", y_col = "label",batch_size =16, shuffle = False, class_mode='categorical',target_size = (299,299))
     elif(testOn=='Casia'):
      print('Loading Casia Samples')     
      test_full = pd.read_csv('//content/drive/MyDrive/ML_Data/Labels/CASIA_FASD_TEST.csv')
      test_folder=casia_test_folder
      
      test_full=test_full.dropna()
      if os.path.exists('/content/CASIA_FASD_FRAMES'):
        print('Found Local Files')
        test_folder='/content/CASIA_FASD_FRAMES/test_release'
        test_full['filename'] = test_full['filename'].str.replace('drive/MyDrive/ML_Data/Datasets_Preprocessed/','')
      if check_every >1 :
        test_full=test_full.iloc[::check_every]
      testData = test_gen.flow_from_dataframe(dataframe =test_full,directory = test_folder,  x_col = "filename", y_col = "label",batch_size =16, shuffle = False, class_mode='categorical',target_size = (299,299))
     elif(testOn=='MSU'):    
      print('Loading MSU Samples')
      test_full = pd.read_csv('/content/drive/MyDrive/ML_Data/Labels/MSU_Test_Labels.csv')
      if os.path.exists('/content/MSU-MFSD_FRAMES'):
        print('Found Local Files')
        test_folder='/content/MSU-MFSD_FRAMES/scene01'
        test_full['filename'] = test_full['filename'].str.replace('drive/MyDrive/ML_Data/Datasets_Preprocessed/','')

      else:
        test_folder='/content/drive/MyDrive/ML_Data/Datasets_Preprocessed/MSU-MFSD_FRAMES/scene01'

      test_full=test_full.dropna()
      if check_every >1 :
        test_full=test_full.iloc[::check_every]
      
      testData = test_gen.flow_from_dataframe(dataframe =test_full,directory = test_folder,  x_col = "filename", y_col = "label",batch_size =16, shuffle = False, class_mode='categorical',target_size = (299,299))
     
     preds=l_model.predict(testData,  verbose = 1, use_multiprocessing = True, workers = 16)
     
     p=testData.filenames
     lab=testData.labels
     
     DF = pd.DataFrame(preds)
     DF['filenames']=p
     DF['labels_orig']=lab
     print('Results Done '+"./Results/"+filename+'.csv')
     DF.to_csv("./Results/"+filename+'.csv')
     return DF
def SetUp_Training(dataset='Replay',check_every=1):
  
  if(dataset=='Replay'):
      print('Loading Replay Samples')
      train_folder=Replay_train_folder
      val_folder=Replay_val_folder
     
      train_full = Replay_train_full
      dev_full = Replay_dev_full
      if os.path.exists('/content/replay_data_every_10th'):
         print('Found Local Files')
         train_folder='/content/replay_data_every_10th/train'
         val_folder='/content/replay_data_every_10th/dev'
         train_full['filename'] = train_full['filename'].str.replace('drive/MyDrive/ML_Data/Datasets_Preprocessed/','')
         dev_full['filename'] = dev_full['filename'].str.replace('drive/MyDrive/ML_Data/Datasets_Preprocessed/','')

 
      train_1=train_full
      dev_1=dev_full.iloc[::]
  elif(dataset=='Casia'):
     print('Loading Casia Samples')

     train_folder=Casia_train_folder
     val_folder=Casia_val_folder
     dev_full = Casia_dev_full
     train_full = Casia_train_full
     if os.path.exists('/content/CASIA_FASD_FRAMES'): 
         print('Found Local Files')
         train_folder='/content/CASIA_FASD_FRAMES/train_release'
         train_full['filename'] = train_full['filename'].str.replace('drive/MyDrive/ML_Data/Datasets_Preprocessed/','')
         dev_full['filename'] = dev_full['filename'].str.replace('drive/MyDrive/ML_Data/Datasets_Preprocessed/','')
         val_folder=train_folder
  # for half files
     train_1=train_full
     dev_1=dev_full.iloc[::]
  elif(dataset=='MSU'):    
      print('Loading MSU Samples')
      train_full = pd.read_csv('/content/drive/MyDrive/ML_Data/Labels/MSU_Test_Labels.csv')# for half files
      train_folder='/content/drive/MyDrive/ML_Data/Datasets_Preprocessed/MSU-MFSD_FRAMES'
      if os.path.exists('/content/CASIA_FASD_FRAMES'): 
         print('Found Local Files')
         train_folder='/content/MSU-MFSD_FRAMES'
         train_full['filename'] = train_full['filename'].str.replace('drive/MyDrive/ML_Data/Datasets_Preprocessed/','')
         
      train_1=train_full[train_full.index % 4 != 0]
      val_folder=train_folder
      dev_1=train_full.iloc[3::4]
  print('Training folder : '+train_folder)
  print('Val folder : '+val_folder)
  if check_every >1 :
        train_1=train_1.iloc[::check_every]
        dev_1=dev_1.iloc[::check_every]
      
  train_data_1 = train_gen.flow_from_dataframe(dataframe = train_1,directory = train_folder,  x_col = "filename", y_col = "label",batch_size =16, shuffle = True, class_mode='categorical',target_size = (299,299))
  val_data_1   = val_gen.flow_from_dataframe(dataframe = dev_1, directory = val_folder, x_col = 'filename', y_col = 'label',batch_size = 16, shuffle = False, class_mode= 'categorical',target_size = (299,299))
  return train_data_1,val_data_1     


In [ ]:
def getModel_ELBP_Ablation(Xweights='Replay',colors='RGB',images=False,xception_trainable=False,L1=True,L2=False,check_mode=False):
    print('Loading xception weights')
    
    if(Xweights=='Replay'):
      print('Loading xception weights -Replay')
      loaded_model=keras.models.load_model('/content/drive/MyDrive/ML_Data/SavedModels/Xcept_GAP_Replay_9.hdf5' )
   
    elif(Xweights=='Casia'):
      print('Loading xception weights - Casia')      
      loaded_model=keras.models.load_model('/content/drive/MyDrive/ML_Data/SavedModels/Xcept_GAP_Casia_7.hdf5')
   
    elif(Xweights=='xcept'):
      print('Loading xception weights - imageNet')      
      loaded_model=SE_Xception()
   
    elif(Xweights=='MSU'):
      print('Loading xception weights - MSU')      
      loaded_model=keras.models.load_model('/content/drive/MyDrive/ML_Data/SavedModels/Xcept_GAP_MSU_10.hdf5')
   


    else:
      print('Invalid Fresh Weights')
      return
    inputs_s = keras.Input(shape=(299, 299, 3))
    inputs_s2 = keras.Input(shape=(299, 299, 3))
    print('Slicing model for features')
    X_model= Model(inputs=loaded_model.input, outputs=loaded_model.layers[-2].output,name='Squeeze_Xception')
    Y_model= Model(inputs=loaded_model.input, outputs=loaded_model.layers[-2].output,name='Squeeze_Xception_Y')
    x_out=X_model(inputs_s2,training=False)
      
      #common layer to get face data . this will pass the colored cropped face to ELBP and HSV generator
    faceGetter=FacePlate(input_dim=(299,299,1))(inputs_s2)
    iLBp=keras.Input(shape=(faceGetter.shape[1],faceGetter.shape[2],faceGetter.shape[3]))
    R_=tf.expand_dims(iLBp[:,:,:,0],-1)
    G_=tf.expand_dims(iLBp[:,:,:,1],-1)
    B_=tf.expand_dims(iLBp[:,:,:,2],-1)
    R_i=tf.expand_dims(inputs_s2[:,:,:,0],-1)
    G_i=tf.expand_dims(inputs_s2[:,:,:,1],-1)
    B_i=tf.expand_dims(inputs_s2[:,:,:,2],-1)
    HSV=tf.image.rgb_to_hsv(iLBp)
    V_=tf.expand_dims(HSV[:,:,:,2],-1)    
    S_=tf.expand_dims(HSV[:,:,:,1],-1)    
    H_=tf.expand_dims(HSV[:,:,:,0],-1)    

    #Face
    Y_=tf.keras.layers.Add()([Lambda(lambda x: x * 0.299)(R_),Lambda(lambda x: x * 0.587)(G_),Lambda(lambda x: x * 0.114)(B_)])
    Cb_=tf.keras.layers.Add()([Lambda(lambda x: x * (-0.168736))(R_),Lambda(lambda x: x * (-0.331264))(G_),Lambda(lambda x: x * 0.5)(B_)])
    Cr_=tf.keras.layers.Add()([Lambda(lambda x: x *0.5)(R_),Lambda(lambda x: x * (-0.418688))(G_),Lambda(lambda x: x * (-0.081312))(B_)])
    Y_i=tf.keras.layers.Add()([Lambda(lambda x: x * 0.299)(R_i),Lambda(lambda x: x * 0.587)(G_i),Lambda(lambda x: x * 0.114)(B_i)])
    Cb_i=tf.keras.layers.Add()([Lambda(lambda x: x * (-0.168736))(R_i),Lambda(lambda x: x * (-0.331264))(G_i),Lambda(lambda x: x * 0.5)(B_i)])
    Cr_i=tf.keras.layers.Add()([Lambda(lambda x: x *0.5)(R_i),Lambda(lambda x: x * (-0.418688))(G_i),Lambda(lambda x: x * (-0.081312))(B_i)])
    YCbCr=keras.layers.concatenate([Y_i,Cb_i,Cr_i],name='ycbcb')
    rgY=keras.Model(inputs_s2,YCbCr,name='RGB_To_YCbCr')
    print(YCbCr)
    y_out=Y_model(rgY(inputs_s2))
    print(y_out)
    
    R_ELB_out=  Slice_ELBP_Model  (iLBp,R_ ,n='R_'  ,l1=L1,l2=L2)(faceGetter)
    G_ELB_out=  Slice_ELBP_Model  (iLBp,G_ ,n='G_'  ,l1=L1,l2=L2)(faceGetter)
    B_ELB_out=  Slice_ELBP_Model  (iLBp,B_ ,n='B_'  ,l1=L1,l2=L2)(faceGetter)
    Y_ELB_out=  Slice_ELBP_Model  (iLBp,Y_ ,n='Y_'  ,l1=L1,l2=L2)(faceGetter)
    Cb_ELB_out= Slice_ELBP_Model (iLBp,Cb_,n='Cb_',l1=L1 ,l2=L2) (faceGetter)
    Cr_ELB_out= Slice_ELBP_Model (iLBp,Cr_,n='Cr_',l1=L1 ,l2=L2) (faceGetter)
    V_ELB_out = Slice_ELBP_Model(iLBp,V_ ,n='V_' ,l1=L1 ,l2=L2) (faceGetter)
    H_ELB_out = Slice_ELBP_Model(iLBp,H_ ,n='H_' ,l1=L1 ,l2=L2) (faceGetter)
    S_ELB_out = Slice_ELBP_Model(iLBp,S_ ,n='S_' ,l1=L1 ,l2=L2)(faceGetter)
    # connect all
    if colors=='RGB':
      el_c=keras.layers.concatenate([R_ELB_out,G_ELB_out,B_ELB_out],axis=1,name='lbp_concat_all')
    elif colors=='HSV':
      el_c=keras.layers.concatenate([H_ELB_out,S_ELB_out,V_ELB_out],axis=1,name='lbp_concat_all')
    elif colors=='YCbCr':
      el_c=keras.layers.concatenate([Y_ELB_out,Cr_ELB_out,Cb_ELB_out],axis=1,name='lbp_concat_all')
    elif colors=='YCbCr+HSV':
      el_c=keras.layers.concatenate([Y_ELB_out,Cr_ELB_out,Cb_ELB_out,H_ELB_out,S_ELB_out,V_ELB_out],axis=1,name='lbp_concat_all')  
    elif colors=='RGB+HSV':
      el_c=keras.layers.concatenate([R_ELB_out,G_ELB_out,B_ELB_out,H_ELB_out,S_ELB_out,V_ELB_out],axis=1,name='lbp_concat_all')
    elif colors=='RGB+V':
      el_c=keras.layers.concatenate([R_ELB_out,G_ELB_out,B_ELB_out,V_ELB_out],axis=1,name='lbp_concat_all')
    elif colors=='RGB+YCbCr':
      el_c=keras.layers.concatenate([R_ELB_out,G_ELB_out,B_ELB_out,Y_ELB_out,Cr_ELB_out,Cb_ELB_out],axis=1,name='lbp_concat_all')
    elif colors=='RGB+YCbCr+V':
      el_c=keras.layers.concatenate([R_ELB_out,G_ELB_out,B_ELB_out,V_ELB_out,Y_ELB_out,Cr_ELB_out,Cb_ELB_out],axis=1,name='lbp_concat_all')
    else:
      print('Unknown color combo')
      return 0
    
   
    el_c=LayerNormalization()(el_c)
    el_c=keras.layers.Dense(512,activation='relu',name='e_dense_1208')(el_c)
    el_c=keras.layers.Dense(32,activation='relu',name='e_dense_128')(el_c)
    
    
    el_c=keras.layers.concatenate([x_out,el_c])
    el_c=keras.layers.Dense(64,activation='relu',name='f_dense_128')(el_c)

    
    
   

  
    outputs = keras.layers.Dense(2,activation='softmax',name='e_dense_3')(el_c)
    f_model=keras.Model(inputs_s2,outputs)


    
    f_model.compile(optimizer='adam', loss='binary_crossentropy',metrics="categorical_accuracy",steps_per_execution=8)
    if xception_trainable:
      print('xception sub model is trainable!')
    else  :
      print('xception sub model is NOT trainable!')
    f_model.get_layer('Squeeze_Xception').trainable=xception_trainable
    #f_model.get_layer('Squeeze_Xception_Y').trainable=xception_trainable
    if(images):
      model_photu(LBP_container,name='Full_model')
      print('Model image saved to : '+os.getcwd()+'/model_images/Full_model.png')
    return f_model    




In [ ]:
f_model=getModel_ELBP_Ablation(Xweights='Casia',colors='RGB+YCbCr+V',L1=True)


In [ ]:
#Training - Casia

train_data_1,val_data_1=SetUp_Training(dataset='Casia',check_every=1)

f_model.fit(train_data_1,epochs=20, validation_data=val_data_1, verbose = 1,use_multiprocessing=True,workers=16)



In [ ]:
#Testing 

TestModel(f_model,'Replay','R_on_C_RGB_Wide',check_every=5)
print(Parameters_name_list(['R_on_C_RGB_Wide']))
TestModel(f_model,'Casia','C_on_C_RGB_Wide',check_every=5)
print(Parameters_name_list(['C_on_C_RGB_Wide']))

In [ ]:

train_data_1,val_data_1=SetUp_Training(dataset='Replay',check_every=1)

f_model.fit(train_data_1,epochs=20, validation_data=val_data_1, verbose = 1,use_multiprocessing=True,workers=16)


In [ ]:
TestModel(f_model,'Replay','R_on_R_All_Wide',check_every=5)
print(Parameters_name_list(['R_on_R_All_Wide']))
TestModel(f_model,'Casia','C_on_R_All_Wide',check_every=5)
print(Parameters_name_list(['C_on_R_All_Wide']))

In [ ]:
TestModel(f_model,'Replay','R_on_C_RGB_YCbCr_HSV',check_every=4)
print(Parameters_name_list(['R_on_C_RGB_YCbCr_HSV']))

In [ ]:
TestModel(f_model,'Replay','R_on_R_RGB_YCbCr_HSV',check_every=2)
print(Parameters_name_list(['R_on_R_RGB_YCbCr_HSV']))

In [ ]:
df=Parameters_name_list(['R_on_C_RGB_YCbCr_HSV','C_on_C_RGB_YCbCr_L012_2','M_on_C_RGB_YCbCr_L012',
                            'R_on_R_RGB_YCbCr_HSV','M_on_R_YCbCr_L012','C_on_R_RGB_YCbCr_V_L012_2',
                            'C_on_M_RGB_YCbCrV_L012_512','R_on_M_L012H_512_1','M_on_M_L012H_512'])

In [ ]:
df.to_csv('/content/final_Results.csv',index=False)

In [ ]:
df